Polars, a widely used single-machine DataFrame library driven by an OLAP Query Engine, has introduced GPU acceleration starting with the v1.3 release. By integrating with NVIDIA GPUs via the RAPIDS cuDF engine, Polars delivers significantly enhanced performance.

Designed to handle datasets of 10–100+ GB interactively with a single GPU, the GPU engine integrates seamlessly into the Polars Lazy API. To enable it, simply specify `engine="gpu"` in the `collect` operation.

The GPU engine leverages Polars' optimizer for efficient execution and minimal memory consumption. It remains compatible with existing Polars ecosystem tools and offers a smooth CPU fallback for queries not supported on the GPU.

This notebook provides a concise overview of the Polars GPU engine powered by cuDF.

In [1]:
!nvidia-smi | head

Sun Nov 24 06:57:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |


Setup
Installing the Polars GPU Engine
List item
List item
List item
List item
List item
List item
Install Polars with GPU Engine by using a feature flag in the standard pip install command pip install polars[gpu] --extra-index-url=https://pypi.nvidia.com.

Since this notebook may be used in environments with an existing Polars installation (e.g., Google Colab), we'll add the -U flag to upgrade Polars if it's already present.

In [2]:
!pip install -U polars[gpu] --extra-index-url --extra-index-url=https://pypi.anaconda.org/rapidsai-wheels-nightly/simple
!pip install  "cudf_polars_cu12-24.8.0a433-py3-none-any.whl"

Looking in indexes: https://pypi.org/simple, --extra-index-url=https://pypi.anaconda.org/rapidsai-wheels-nightly/simple
  Using cached polars-1.14.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.5/35.5 MB 42.9 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 1.8.2
    Uninstalling polars-1.8.2:
      Successfully uninstalled polars-1.8.2
Processing ./cudf_polars_cu12-24.8.0a433-py3-none-any.whl
cudf-polars-cu12 is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [3]:
!pip install polars==1.5


  Using cached polars-1.5.0-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (14 kB)
Using cached polars-1.5.0-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (31.6 MB)
  Attempting uninstall: polars
    Found existing installation: polars 1.14.0
    Uninstalling polars-1.14.0:
      Successfully uninstalled polars-1.14.0


Install Other Important Dependencies
To use the built-in data visualization capabilities of Polars, you'll need to install a few additional dependencies. We'll also install pynvml to help us determine which dataset size to use.

In [4]:
!pip install hvplot jupyter_bokeh holoviews==1.19 pynvml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 78.7 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
  Attempting uninstall: holoviews
    Found existing installation: holoviews 

We'll be working with a roughly 22GB dataset of simulated financial transactions from Kaggle by default. If you're running this notebook on Google Colab using the T4 GPU in the Colab free tier, we'll download a smaller version of this dataset (about 20% of the size) to fit on the relatively weaker CPU and GPU.

We're downloading a copy of this dataset from a GCS bucket hosted by NVIDIA to provide faster download speeds. We'll start by downloading the data. This should take about 30 seconds.

In [6]:
import pynvml
pynvml.nvmlInit()
pynvml.nvmlDeviceGetName(pynvml.nvmlDeviceGetHandleByIndex(0))
mem = pynvml.nvmlDeviceGetMemoryInfo(pynvml.nvmlDeviceGetHandleByIndex(0))
mem = mem.total/1e9

if mem < 24:
  !wget  https://storage.googleapis.com/rapidsai/polars-demo/transactions-t4-20.parquet -O transactions.parquet

else:
   !wget https://storage.googleapis.com/rapidsai/polars-demo/transactions.parquet -O transactions.parquet

!wget https://storage.googleapis.com/rapidsai/polars-demo/rainfall_data_2010_2020.csv

--2024-11-24 07:04:17--  https://storage.googleapis.com/rapidsai/polars-demo/transactions-t4-20.parquet
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.137.207, 142.251.2.207, 142.250.141.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.137.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 857658422 (818M) [application/octet-stream]
Saving to: ‘transactions.parquet’

transactions.parque 100%[===================>] 817.93M   103MB/s    in 11s     

2024-11-24 07:04:29 (72.0 MB/s) - ‘transactions.parquet’ saved [857658422/857658422]

--2024-11-24 07:04:29--  https://storage.googleapis.com/rapidsai/polars-demo/rainfall_data_2010_2020.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.137.207, 142.251.2.207, 142.250.141.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.137.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131421 (128K) [te

In [7]:
import polars as pl
import hvplot.polars
from polars.testing import assert_frame_equal

In [8]:
pl.__version__

'1.5.0'

In [12]:
transactions = pl.scan_parquet("transactions.parquet")


In [13]:
transactions.collect_schema()

Schema([('CUST_ID', String),
        ('START_DATE', Date),
        ('END_DATE', Date),
        ('TRANS_ID', String),
        ('DATE', Date),
        ('YEAR', Int64),
        ('MONTH', Int64),
        ('DAY', Int64),
        ('EXP_TYPE', String),
        ('AMOUNT', Float64)])

In [14]:
transactions.head(5).collect()

CUST_ID,START_DATE,END_DATE,TRANS_ID,DATE,YEAR,MONTH,DAY,EXP_TYPE,AMOUNT
str,date,date,str,date,i64,i64,i64,str,f64
"""CI6XLYUMQK""",2015-05-01,null,"""T8I9ZB5A6X90UG8""",2015-09-11,2015,9,11,"""Motor/Travel""",20.27
"""CI6XLYUMQK""",2015-05-01,null,"""TZ4JSLS7SC7FO9H""",2017-02-08,2017,2,8,"""Motor/Travel""",12.85
"""CI6XLYUMQK""",2015-05-01,null,"""TTUKRDDJ6B6F42H""",2015-08-01,2015,8,1,"""Housing""",383.8
"""CI6XLYUMQK""",2015-05-01,null,"""TDUHFRUKGPPI6HD""",2019-03-16,2019,3,16,"""Entertainment""",5.72
"""CI6XLYUMQK""",2015-05-01,null,"""T0JBZHBMSVRFMMD""",2015-05-15,2015,5,15,"""Entertainment""",11.06


In [16]:
%%time
transactions.select(pl.col("AMOUNT").sum()).collect()

CPU times: user 1.39 s, sys: 384 ms, total: 1.78 s
Wall time: 949 ms


AMOUNT
f64
3.6183e9


In [18]:
%%time
transactions.select(pl.col("AMOUNT").sum()).collect(engine="gpu")

CPU times: user 260 ms, sys: 33.1 ms, total: 293 ms
Wall time: 283 ms


AMOUNT
f64
3.6183e9


In [20]:
gpu_engine = pl.GPUEngine(
    device=0,
    raise_on_fail = True,
)

In [31]:
%%time
transactions.select(pl.col("AMOUNT").sum()).collect(engine=gpu_engine)

CPU times: user 293 ms, sys: 16.4 ms, total: 310 ms
Wall time: 303 ms


AMOUNT
f64
3.6183e9


In [32]:
%%time

res_cpu = (
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").sum())
    .sort(by="AMOUNT", descending=True)
    .head()
    .collect()
)
res_cpu

CPU times: user 4.22 s, sys: 2.89 s, total: 7.11 s
Wall time: 5.39 s


CUST_ID,AMOUNT
str,f64
"""CA9UYOQ5DA""",2.0290e6
"""CJUK2MTM5Q""",1.8115e6
"""CYXX1NBIKL""",1.8082e6
"""C6ILEYAYQ9""",1.7961e6
"""CCNBC305GI""",1.7274e6


In [34]:
%%time

res_gpu = (
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").sum())
    .sort(by="AMOUNT", descending=True)
    .head()
    .collect(engine=gpu_engine)
)
res_gpu


CPU times: user 361 ms, sys: 3.67 ms, total: 365 ms
Wall time: 356 ms


CUST_ID,AMOUNT
str,f64
"""CA9UYOQ5DA""",2.0290e6
"""CJUK2MTM5Q""",1.8115e6
"""CYXX1NBIKL""",1.8082e6
"""C6ILEYAYQ9""",1.7961e6
"""CCNBC305GI""",1.7274e6


In [35]:
assert_frame_equal(res_cpu, res_gpu)

In [36]:
query = """
SELECT CUST_ID, SUM(AMOUNT) as sum_amt
FROM transactions
GROUP BY CUST_ID
ORDER BY sum_amt desc
LIMIT 5
"""

%time pl.sql(query).collect()
%time pl.sql(query).collect(engine=gpu_engine)

CPU times: user 4.28 s, sys: 2.67 s, total: 6.95 s
Wall time: 4.9 s
CPU times: user 360 ms, sys: 21.9 ms, total: 382 ms
Wall time: 359 ms


CUST_ID,sum_amt
str,f64
"""CA9UYOQ5DA""",2.0290e6
"""CJUK2MTM5Q""",1.8115e6
"""CYXX1NBIKL""",1.8082e6
"""C6ILEYAYQ9""",1.7961e6
"""CCNBC305GI""",1.7274e6


In [37]:
%%time

(
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").max().alias("max_amount"))
    .sort(by="max_amount", descending=True)
    .head()
    .collect(engine=gpu_engine)
)

CPU times: user 385 ms, sys: 3.68 ms, total: 389 ms
Wall time: 380 ms


CUST_ID,max_amount
str,f64
"""CIP0I11MG2""",6201.45
"""C4O38N5TQS""",6077.49
"""CL2M3N3K90""",6041.59
"""CC472PU9O8""",5929.14
"""CGR8UI27OK""",5903.61


In [38]:
(
    transactions
    .filter(pl.col("CUST_ID") == "CIP0I11MG2")
    .select(pl.col("AMOUNT").max())
    .collect()
)

AMOUNT
f64
6201.45


In [39]:
res, prof = (
    transactions
    .filter(pl.col("CUST_ID") == "CIP0I11MG2")
    .select(pl.col("AMOUNT").max())
    .profile()
)

prof.with_columns(
    ((pl.col("end") - pl.col("start")) / pl.col("end").max() * 100)
    .alias("pct_time_spent")
)


node,start,end,pct_time_spent
str,u64,u64,f64
"""optimization""",0,9,0.000782
"""parquet(transactions.parquet, …",9,1150569,99.996524
"""select(AMOUNT)""",1150576,1150600,0.002086


In [40]:
%%time

res = (
    transactions
    .group_by(["EXP_TYPE", "YEAR", "MONTH"])
    .agg(pl.mean("AMOUNT"))
    .sort(["EXP_TYPE", "YEAR", "MONTH"])
    .collect()
)

CPU times: user 11 s, sys: 8.26 s, total: 19.3 s
Wall time: 13.6 s


In [41]:
%%time

res = (
    transactions
    .group_by(["EXP_TYPE", "YEAR", "MONTH"])
    .agg(pl.mean("AMOUNT"))
    .sort(["EXP_TYPE", "YEAR", "MONTH"])
    .collect(engine=gpu_engine)
)

CPU times: user 467 ms, sys: 84.4 ms, total: 551 ms
Wall time: 1.65 s


In [42]:
res = res.with_columns(
    pl.datetime(pl.col("YEAR"), pl.col("MONTH"), day=1)
    .alias("year-mon")
)

res.hvplot.scatter(x="year-mon", y="AMOUNT", by="EXP_TYPE")

:NdOverlay   [EXP_TYPE]
   :Scatter   [year-mon]   (AMOUNT)

In [43]:
res

EXP_TYPE,YEAR,MONTH,AMOUNT,year-mon
str,i64,i64,f64,datetime[μs]
"""Bills and Utilities""",2010,1,85.415487,2010-01-01 00:00:00
"""Bills and Utilities""",2010,2,90.495608,2010-02-01 00:00:00
"""Bills and Utilities""",2010,3,94.857936,2010-03-01 00:00:00
"""Bills and Utilities""",2010,4,95.238054,2010-04-01 00:00:00
"""Bills and Utilities""",2010,5,94.464738,2010-05-01 00:00:00
…,…,…,…,…
"""Tax""",2020,8,233.777692,2020-08-01 00:00:00
"""Tax""",2020,9,234.608419,2020-09-01 00:00:00
"""Tax""",2020,10,234.351233,2020-10-01 00:00:00


In [44]:
names = ['Location', 'Rainfall (inches)', 'Date', 'YEAR', 'MONTH', 'DAY']

weather = pl.scan_csv("rainfall_data_2010_2020.csv", new_columns=names)
weather.head().collect()

Location,Rainfall (inches),Date,YEAR,MONTH,DAY
str,f64,i64,i64,i64,i64
"""Tatooine""",0.33,20100101,2010,1,1
"""Tatooine""",0.0,20100102,2010,1,2
"""Tatooine""",0.28,20100103,2010,1,3
"""Tatooine""",0.26,20100104,2010,1,4
"""Tatooine""",0.39,20100105,2010,1,5


In [45]:
weather_cleaned = (
    weather
    .with_columns(pl.col("Date").cast(pl.Utf8).str.strptime(pl.Date(), "%Y%m%d"))
    .collect()
)

In [46]:
%%time

(
    transactions
    .join(
        other=weather_cleaned.lazy(),
        left_on="DATE",
        right_on="Date",
        how="inner"
    )
    .group_by(["EXP_TYPE", "DATE"])
    .agg(pl.mean("Rainfall (inches)"))
    .sort(["DATE", "EXP_TYPE", "Rainfall (inches)"])
    .head()
    .collect()
)

CPU times: user 12.7 s, sys: 3.91 s, total: 16.6 s
Wall time: 11.5 s


EXP_TYPE,DATE,Rainfall (inches)
str,date,f64
"""Bills and Utilities""",2010-01-01,0.33
"""Clothing""",2010-01-01,0.33
"""Education""",2010-01-01,0.33
"""Entertainment""",2010-01-01,0.33
"""Fines""",2010-01-01,0.33
